In [17]:
%matplotlib inline
import seaborn as sns; #sns.set(style="ticks", color_codes=True)
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import scipy
style.use('fast')
from datetime import datetime
from io import StringIO
import re
import numpy as np
import os
from tqdm import tqdm
import json as js
import codecs
import subprocess


In [4]:

sns.set_style('ticks',{'legend.frameon':True} )
sns.set_palette('colorblind')
FIG_SIZE = (12, 3)
font = {'family' : 'Times New Roman',
        'weight' : 'bold',
        'size'   : 22}
plt.rc('font', **font)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['text.usetex'] = True

plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 22
plt.rcParams['axes.labelsize']= 22


# Packages and Vulnerabilities

In [131]:
official_layers=pd.read_csv('../../docker/csv/layers_official_23MARCH.csv', 
                            sep=';', dtype=object, index_col=None,  error_bad_lines=False,
                           usecols=['slug','tag','created']).drop_duplicates()
official_layers=(official_layers.
                 #query('slug=="node"').
                 assign(created = lambda x: x['created'].apply(lambda d: d.split('T')[0]))
                )
official_layers['tag']=official_layers['slug']+':'+official_layers['tag']
official_layers.head()

,slug,tag,created
0,tomcat,tomcat:8.0.37-jre7,2016-09-23
11,postgres,postgres:9.5.12-alpine,2018-03-02
20,crate,crate:0.56,2016-11-21
28,traefik,traefik:v1.4.0-rc5,2017-10-10
30,gradle,gradle:4.5.0-jre-alpine,2018-01-26


In [212]:
docker_package=pd.read_csv('../../npm_docker/csv/packages_from_anchor.csv', sep=';', dtype=object, index_col=None,  error_bad_lines=False)
docker_package['tag']=docker_package['slug']+':'+docker_package['tag']
docker_package.drop_duplicates(inplace=True)
print(len(docker_package))
docker_package.head(2)

296699


,slug,tag,package,version,base_os,base_od_version
0,ghost,ghost:0,once,1.4.0,debian,8
1,ghost,ghost:0,validator,8.2.0,debian,8


In [213]:
docker_package=(docker_package.
                merge(
                 official_layers,
                 left_on='tag', 
                 right_on='tag', 
                    how='left').
             dropna().
             drop_duplicates()
            )
docker_package.drop(['slug_x','slug_y','base_od_version'], axis=1, inplace=True)
len(docker_package)
docker_package.head()

,tag,package,version,base_os,created
0,ghost:0,once,1.4.0,debian,2018-03-16
1,ghost:0,validator,8.2.0,debian,2018-03-16
2,ghost:0,minimatch,3.0.3,debian,2018-03-16
3,ghost:0,oauth-sign,0.8.2,debian,2018-03-16
4,ghost:0,form-data,2.1.1,debian,2018-03-16


In [134]:
vulnerabilities=pd.read_csv('../../npm_docker/csv/vulnerabilities.csv', sep=';', dtype=object, index_col=None,  error_bad_lines=False)
print(len(vulnerabilities))
vulnerabilities.head(2)

1099


,id,name,package,published,severity,versions,disclosed
0,npm:http-fetch-cookies:20180502,Malicious Package,http-fetch-cookies,"03 May, 2018",high,*,"02 May, 2018"
1,npm:getcookies:20180502,Malicious Package,getcookies,"03 May, 2018",high,*,"02 May, 2018"


we filter only those that are vulnerable

In [139]:
mask = docker_package['package'].isin(vulnerabilities['package'].values)
vul_packages=docker_package[mask]
len(vul_packages)

27750

In [136]:
vul_packages['tag'].drop_duplicates().size

961

In [137]:
package_vul=(docker_package.
             merge(
                 vulnerabilities, how='left', left_on='package', right_on='package').
             dropna().
             drop_duplicates()
            )
print(len(package_vul))
package_vul.head()

40792


,tag,package,version,base_os,created,id,name,published,severity,versions,disclosed
1,ghost:0,validator,8.2.0,debian,2018-03-16,npm:validator:20180218,Regular Expression Denial of Service (ReDoS),"18 Feb, 2018",low,<9.4.1,"18 Feb, 2018"
2,ghost:0,validator,8.2.0,debian,2018-03-16,npm:validator:20160218,Buffer Overflow,"18 Feb, 2018",medium,<5.0.0,"18 Feb, 2016"
3,ghost:0,validator,8.2.0,debian,2018-03-16,npm:validator:20150313,Cross-site Scripting (XSS),"30 Jan, 2017",medium,<3.34.0 >=3.0.0,"12 Mar, 2015"
4,ghost:0,validator,8.2.0,debian,2018-03-16,npm:validator:20130705,Regular Expression Denial of Service (DoS),"12 Nov, 2014",high,<3.22.1,"12 Nov, 2014"
5,ghost:0,validator,8.2.0,debian,2018-03-16,npm:validator:20130705-1,Cross-site Scripting (XSS),"27 Oct, 2014",medium,<2.0.0,"27 Oct, 2014"


In [140]:
uniq_packages=package_vul[['package','version','versions']].drop_duplicates()
len(uniq_packages)

1061

In [141]:
uniq_packages['affected']=uniq_packages.apply(lambda d: affected(d['versions'],d['version']), axis=1)

In [142]:
package_vul=(package_vul.set_index(['package','version','versions']).
             merge(
                 uniq_packages.
                 set_index(['package','version','versions']),
                 left_index=True, 
                 right_index=True, 
                 how='left').
   reset_index().
   dropna().
   drop_duplicates()
  )

In [143]:
len(package_vul.query('affected==True'))

15955

In [255]:
package_vul.to_csv('../../npm_docker/csv/affected_packages.csv',sep=';', index=False)

# Prepare the dataframe of all packages

In [214]:
unique_packages=docker_package[['package','version','created']].drop_duplicates()
print(len(unique_packages))
unique_packages.head(2)

92600


,package,version,created
0,once,1.4.0,2018-03-16
1,validator,8.2.0,2018-03-16


In [215]:
npm_versions=pd.read_csv('../../jsep2018/versions_npm_march.csv', sep=';', dtype=object, index_col=None,  error_bad_lines=False)
print(len(npm_versions))
npm_versions.head(2)

4760450


,package,version,date,release_type
0,0,0.0.0,2014-04-01,initial
1,0-,0.0.1,2017-03-06,initial


In [216]:
mask=npm_versions.package.isin(unique_packages.package)
npm_versions=npm_versions[mask]
npm_versions=npm_versions.sort_values(['package','date'],ascending=True)
len(npm_versions)

30348

In [217]:
unique_packages['latest']=unique_packages.apply(lambda d: latest_version_date(d['package'],
                                                                              d['created']),
                                                axis=1
                                               )

In [220]:
unique_packages=unique_packages.query('latest!="undefined"')

In [228]:
unique_packages=(unique_packages.set_index(['package','version']).
             merge(
                 npm_versions.
                 set_index(['package','version']),
                 left_index=True, 
                 right_index=True, 
                 how='left').
   reset_index().
   dropna().
   drop_duplicates()
  )

In [238]:
unique_packages=(docker_package.set_index(['package','latest']).
             merge(
                 npm_versions.
                 rename(columns={'version':'latest','date':'date_latest'}).
                 drop('release_type',axis=1).
                 set_index(['package','latest']),
                 left_index=True, 
                 right_index=True, 
                 how='left').
   reset_index().
   dropna().
   drop_duplicates()
  )

In [ ]:
unique_packages['version_lag']=unique_packages.apply(lambda d: count_versionlag(d['package'],d['date'],
                                                                              d['date_latest']),
                                                axis=1
                                               )


In [ ]:
unique_packages['extraction_date']='2018-03-13'
unique_packages['version_lag_today']=unique_packages.apply(lambda d: count_versionlag(d['package'],d['date'],
                                                                              d['extraction_date']),
                                                axis=1
                                               )


In [318]:
t=(docker_package.set_index(['package','version','created']).
             merge(
                 unique_packages.
                 set_index(['package','version','created']),
                 left_index=True, 
                 right_index=True, 
                 how='left').
   reset_index().
   dropna().
   drop_duplicates()
  )
len(t)

290031

In [328]:
t.to_csv('../../npm_docker/csv/docker_packages_timelag.csv', index=False, sep=";")

In [327]:
t

,package,version,created,tag,base_os,latest,date,release_type,date_latest,version_lag,extraction_date,version_lag_today
7,1to2,1.0.0,2016-08-01,ghost:0.8,debian,1.0.0,2016-04-19,initial,2016-04-19,"0,0,0",2018-03-13,"0,0,0"
8,1to2,1.0.0,2016-09-28,ghost:0.11.0,debian,1.0.0,2016-04-19,initial,2016-04-19,"0,0,0",2018-03-13,"0,0,0"
9,1to2,1.0.0,2016-10-05,ghost:0.11.1,debian,1.0.0,2016-04-19,initial,2016-04-19,"0,0,0",2018-03-13,"0,0,0"
10,1to2,1.0.0,2017-03-01,ghost:0.11.5,debian,1.0.0,2016-04-19,initial,2016-04-19,"0,0,0",2018-03-13,"0,0,0"
11,1to2,1.0.0,2017-04-20,ghost:0.11.7-alpine,alpine,1.0.0,2016-04-19,initial,2016-04-19,"0,0,0",2018-03-13,"0,0,0"
12,1to2,1.0.0,2017-08-03,ghost:1.0-alpine,alpine,1.0.0,2016-04-19,initial,2016-04-19,"0,0,0",2018-03-13,"0,0,0"
13,1to2,1.0.0,2017-08-03,ghost:1.1-alpine,alpine,1.0.0,2016-04-19,initial,2016-04-19,"0,0,0",2018-03-13,"0,0,0"
14,1to2,1.0.0,2017-08-03,ghost:1.2-alpine,alpine,1.0.0,2016-04-19,initial,2016-04-19,"0,0,0",2018-03-13,"0,0,0"
15,1to2,1.0.0,2017-08-03,ghost:1.3-alpine,alpine,1.0.0,2016-04-19,initial,2016-04-19,"0,0,0",2018-03-13,"0,0,0"
16,1to2,1.0.0,2017-08-03,ghost:1.4-alpine,alpine,1.0.0,2016-04-19,initial,2016-04-19,"0,0,0",2018-03-13,"0,0,0"


# Helpers

In [297]:

RE_SEMVER = r'^(?:v|V)?(?P<v_major>\d+)\.(?P<v_minor>\d+)\.(?P<v_patch>\d+)(?P<v_misc>.*)$'

def count_versionlag(package,date,date_latest):
    if date==date_latest:
        return '0,0,0'
    tmp=npm_versions.query('package=="'+package+'" and date>"'+date+'"').query('date<="'+date_latest+'"')
    tmp=tmp.groupby('release_type').count()
    try:
        major=str(tmp.package['major'])
    except:
        major="0"
        
    try:
        minor=str(tmp.package['minor'])
    except:
        minor="0"
        
    try:
        patch=str(tmp.package['patch'])
    except:
        patch="0"
        
    return major+','+minor+','+patch
    

def latest_version_date(package,date):
    try:
        return npm_versions.query('package=="'+package+'" and date<="'+date+'"').values[-1][1]
    except:
        return 'undefined'

def affected(constraint,version):
    verss=[]
    verss.append(version)
    return len(semver(constraint,verss)) > 0

def semver(constraint, versions):
    """
    Return the versions in ``versions'' that satisfy given constraint.
    Semantic is provided by the semver tool of nodejs.
    """
    args = ['semver', '-r', constraint] + list(versions)
    
    completed = subprocess.run(args, stdout=subprocess.PIPE)
    if completed.returncode == 0:
        return completed.stdout.decode().strip().split('\n')
    else:
        return []
    

def comply_semver(series_of_releases):
    extracted = series_of_releases.str.extract(RE_SEMVER, expand=True)
    return (
        extracted
        [['v_major', 'v_minor', 'v_patch']]
        .astype('str')
        .assign(Semver=lambda d: d['v_major'] + '.' + d['v_minor'] + '.' + d['v_patch'])
        ['Semver']
    )
    

def kind_of_update(previous, current, allow_misc=False):
    if previous is pandas.np.nan or previous is pandas.NaT:
        return 'initial'
    
    semver_previous = re.match(RE_SEMVER, previous)
    semver_current = re.match(RE_SEMVER, current)
    
    previous_v = semver_previous.groupdict()
    current_v = semver_current.groupdict()
    
    if previous_v['v_major'] != current_v['v_major']:
        return 'major'
    elif previous_v['v_minor'] != current_v['v_minor']:
        return 'minor'
    elif previous_v['v_patch'] != current_v['v_patch']:
        return 'patch'
    elif previous_v['v_misc'] != current_v['v_misc']:
        return 'misc' if allow_misc else 'patch'
    else:
        raise ValueError('The two versions are similar from a semantic point of view')
        
